This is somewhat modified implementation of gradient free black box adversarial attack approach from https://arxiv.org/abs/1805.11090 (for now the official repository is empty, but here's the link https://github.com/nesl/adversarial_genattack) for the competition (https://competitions.codalab.org/competitions/19090) organized as a part of Machines Can See 2018 summit (http://machinescansee.com/).

You can find the baseline code for the competition here https://github.com/AlexanderParkin/MCS2018.Baseline. I used some of it for image preprocessing and saving adversarial examples.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd

from PIL import Image
from showprogress import showprogress

import MCS2018

## Control GPU usage

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Load the black box model before torch (doesn't work otherwise for some reason) :C

You can find details of the contest here https://competitions.codalab.org/competitions/19090.

We first need to load the black box model

In [ ]:
# ! wget http://mcs2018-competition.visionlabs.ru/distribs/cuda9/ubuntu/MCS2018.cpython-36m-x86_64-linux-gnu.so

In [ ]:
gpu_id = 0
net = MCS2018.Predictor(gpu_id)

In [ ]:
import torch

from genattack import *
from torchvision import transforms

# Load datalists

Then load the data used in the contest

In [ ]:
# ! python downloader.py --root /data --main_imgs --student_model_imgs --submit_list --pairs_list

also please unzip the data if necessary

In [ ]:
img_pairs = pd.read_csv('/data/mcs/pairs_list.csv')
img_pairs[:2]

## Define transforms

In [ ]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
REVERSE_MEAN = [-0.485, -0.456, -0.406]
REVERSE_STD = [1/0.229, 1/0.224, 1/0.225]

ATTACK_DIR = '/data/mcs/attack_imgs/'

transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize((112,112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=MEAN, std=STD)
    ])

## Hyperparams

In [ ]:
n_imgs = 5
dim = 512  # descriptor dim
nchannels = 3
h = 112
w = 112
N = 6  # size of population to evolve
G = 500  # number of generations to evolve through
p = torch.cuda.FloatTensor([0.005])
alpha = torch.cuda.FloatTensor([1.])
delta = torch.cuda.FloatTensor([0.05])

## The following 2 functions are taken from original baseline repo to save adversarial images

In [ ]:
def reverse_normalize(tensor, mean, std):
    '''reverese normalize to convert tensor -> PIL Image'''
    tensor_copy = tensor.clone()
    for t, m, s in zip(tensor_copy, mean, std):
        t.div_(s).sub_(m)
    return tensor_copy


def tensor2img(tensor, on_cuda=True):
    tensor = reverse_normalize(tensor, REVERSE_MEAN, REVERSE_STD)
    # clipping
    tensor[tensor > 1] = 1
    tensor[tensor < 0] = 0
    tensor = tensor.squeeze(0)
    if on_cuda:
        tensor = tensor.cpu()
    return transforms.ToPILImage()(tensor)

## Ok, go!

In [ ]:
low_ssim = []  # for images with low ssim
scores = []

In [ ]:
for idx in showprogress(img_pairs.index.values):
    try:
        pairs = {'source': img_pairs.loc[idx].source_imgs.split('|'),
                 'target': img_pairs.loc[idx].target_imgs.split('|')}
        source_img_names = pairs['source']
        target_img_names = pairs['target']

        targets = torch.cuda.FloatTensor(n_imgs, dim)
        for source_img_name in source_img_names:
            source_img_name = os.path.join('/data/mcs/imgs/', source_img_name)
            source_img = Image.open(source_img_name)

            x = transform(source_img)
            x = x.cuda(async=True)
            tavg = torch.cuda.FloatTensor(dim)
            # since the task is to confuse black box between two identities,
            # each having n_imgs images, we simply take average target descriptors
            for i, target_img_name in enumerate(target_img_names):
                target_img_name = os.path.join('/data/mcs/imgs/', target_img_name)
                target_img = Image.open(target_img_name)

                t = transform(target_img).unsqueeze(0).numpy()
                targets[i] = torch.cuda.FloatTensor(net.submit(t))
                tavg += targets[i]
            tavg /= torch.norm(tavg)  # make avg descriptor of unit length

            Pc = attack(x, tavg, delta, alpha, p, N, G, net)

            ssimm = ssim(x.squeeze().permute(1,2,0).cpu().numpy(),
                         Pc[0].permute(1,2,0).cpu().numpy(),
                         multichannel=True)

            d_adv = net.submit(Pc[0][None, :, :, :].cpu().numpy())
            # compute L2 distances between target and adversarial descriptors
            for i in range(n_imgs):
                scores.append(np.linalg.norm(targets[i].cpu().numpy() - d_adv))
            print(sum(scores[-5:]) / 5)  # print the mean score for the current source example

            if ssimm < 0.95:
                print('SSIM low: %s' % source_img_name)
                low_ssim.append(source_img_name)
                continue  # do not save images with low ssim, better retry for them after

            # save adversarial example
            attack_img = tensor2img(Pc[0], True)
            attack_img.save(ATTACK_DIR + os.path.basename(source_img_name).replace('.jpg', '.png'))
    except Exception as e:
        print(source_img_name)
        print(e)
        pass
    

In [ ]:
print(np.array(scores).mean())